In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/preprocessed/newdf2.csv
/kaggle/input/preprocessed/newtest.csv
/kaggle/input/its-a-fraud/train.csv
/kaggle/input/its-a-fraud/test.csv
/kaggle/input/its-a-fraud/mock_submission.csv


In [8]:
dtr=pd.read_csv('../input/preprocessed/newdf2.csv')
dte=pd.read_csv('../input/preprocessed/newtest.csv')

In [9]:
test=pd.read_csv('../input/its-a-fraud/test.csv')

In [10]:
dtr.drop(['Unnamed: 0'],axis=1,inplace=True)
dte.drop(['Unnamed: 0'],axis=1,inplace=True)

In [28]:
X_train=dtr.drop(['isFraud'],axis=1,inplace=False)
y_train=dtr['isFraud']

In [29]:
X_full=X_train
y_full=y_train

In [12]:
X_test=dte.copy()

Using default parameters

In [13]:
from sklearn.tree import DecisionTreeClassifier

In [14]:
clf = DecisionTreeClassifier()
clf.fit(X_train,y_train)

DecisionTreeClassifier()

In [9]:
output = clf.predict(X_test)

In [11]:
from collections import Counter
Counter(output)

Counter({0: 140069, 1: 7566})

In [12]:
dte['isFraud']=output
neww=dte[['TransactionID','isFraud']]
test = test.merge(neww, how='outer',copy=False, on ='TransactionID' )
output1 = test['isFraud']
output1.columns=['Id','isFraud']
output1=pd.DataFrame(output1)
output1.to_csv('output.csv',index=True,index_label='Id')

Using class_wieght="balanced"

In [15]:
clf1 = DecisionTreeClassifier(class_weight="balanced")
clf1.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced')

In [16]:
output1 = clf1.predict(X_test)

In [19]:
from collections import Counter
Counter(output1)

Counter({0: 142483, 1: 5152})

In [20]:
dte['isFraud']=output1
neww=dte[['TransactionID','isFraud']]
test = test.merge(neww, how='outer',copy=False, on ='TransactionID' )
output2 = test['isFraud']
output2.columns=['Id','isFraud']
output2=pd.DataFrame(output1)
output2.to_csv('output.csv',index=True,index_label='Id')

The score using default parameters is 0.708<br>
Using class weight balanced,it is 0.684<br>
We will try to tune the hyperparameters

In [25]:
from sklearn.model_selection import train_test_split 
from sklearn.metrics import roc_auc_score

x_train,x_test,y_train,y_test = train_test_split(X_train,y_train,test_size=0.33, random_state=42)

In [27]:
max_depth = [3,5,7,9,11,13,15]
max_features = ['auto','sqrt','log2']

for depth in max_depth:
    for feature in max_features:
        print('*******max_depth=',depth,', max_feature=',feature)
        clf = DecisionTreeClassifier(max_depth=depth,max_features=feature)
        clf.fit(x_train,y_train)
        y_train_pred=clf.predict_proba(x_train)
        y_test_pred = clf.predict_proba(x_test)
        print('train auc:',roc_auc_score(y_train,y_train_pred[:,1]))
        print('test auc:',roc_auc_score(y_test,y_test_pred[:,1]))
        print('======================================================')

*******max_depth= 3 , max_feature= auto
train auc: 0.6032425309538847
test auc: 0.6074703940061216
*******max_depth= 3 , max_feature= sqrt
train auc: 0.6371959997775309
test auc: 0.6373457678521482
*******max_depth= 3 , max_feature= log2
train auc: 0.6836689629821855
test auc: 0.6914568079294072
*******max_depth= 5 , max_feature= auto
train auc: 0.7064434064396624
test auc: 0.7038268258904281
*******max_depth= 5 , max_feature= sqrt
train auc: 0.6618960622102457
test auc: 0.6629765499853169
*******max_depth= 5 , max_feature= log2
train auc: 0.7624927576942009
test auc: 0.7644561926599972
*******max_depth= 7 , max_feature= auto
train auc: 0.8166160727225611
test auc: 0.8092240788514018
*******max_depth= 7 , max_feature= sqrt
train auc: 0.7971709851783557
test auc: 0.7932507553018334
*******max_depth= 7 , max_feature= log2
train auc: 0.79612011354279
test auc: 0.7986014770236074
*******max_depth= 9 , max_feature= auto
train auc: 0.821729974580474
test auc: 0.8115576580263646
*******max_de

Trying max_depth=13 and max_feature=log2

In [31]:
clf1 = DecisionTreeClassifier(max_depth=13,max_features='log2')
clf1.fit(X_full,y_full)

DecisionTreeClassifier(max_depth=13, max_features='log2')

In [34]:
dte.drop('isFraud',axis=1,inplace=True)

In [35]:
y_test1_pred = clf1.predict(dte)

In [36]:
Counter(y_test1_pred)

Counter({0: 146292, 1: 1343})

In [39]:
dte['isFraud']=y_test1_pred
neww=dte[['TransactionID','isFraud']]
test = test.merge(neww, how='outer',copy=False, on ='TransactionID' )
output2 = dte['isFraud']
output2.columns=['Id','isFraud']
output2=pd.DataFrame(output2)
output2.to_csv('output2.csv',index=True,index_label='Id')

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:9203: FutureWarning: Passing 'suffixes' which cause duplicate columns {'isFraud_x'} in the result is deprecated and will raise a MergeError in a future version.
  validate=validate,


Score=0.55847

In [40]:
clf1 = DecisionTreeClassifier(max_depth=9,max_features='auto')
clf1.fit(X_full,y_full)

DecisionTreeClassifier(max_depth=9, max_features='auto')

In [41]:
dte.drop('isFraud',axis=1,inplace=True)

In [42]:
y_test1_pred = clf1.predict(dte)

In [43]:
Counter(y_test1_pred)

Counter({0: 146132, 1: 1503})

In [45]:
dte['isFraud']=y_test1_pred
neww=dte[['TransactionID','isFraud']]
test = test.merge(neww, how='outer',copy=False, on ='TransactionID' )
output2 = dte['isFraud']
output2.columns=['Id','isFraud']
output2=pd.DataFrame(output2)
output2.to_csv('output3.csv',index=True,index_label='Id')

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:9203: FutureWarning: Passing 'suffixes' which cause duplicate columns {'isFraud_x'} in the result is deprecated and will raise a MergeError in a future version.
  validate=validate,


Score=0.6215